In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [2]:
import keras

from keras.models import Sequential
from keras.models import load_model

from keras.layers import Flatten, Dense
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import pooling


from keras.optimizers import SGD
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint
from sklearn import model_selection
import tensorflow as tf


In [3]:
from PIL import Image
import sys

In [4]:
import cv2 
import glob


In [5]:
dataset = ('C:/Users/Ketan/Thesis/self-driving-car/challenges/challenge-2')

In [6]:
# Read the data
df = pd.io.parsers.read_csv(os.path.join(dataset, 'out.csv'))
# # Split data into training and validation sets
df_train, df_val = model_selection.train_test_split(df, test_size=.2)

In [7]:
df_train.head()

,frame_id,angle,public,torque,speed,File
88,1.480000e+18,-0.095993,0,NaN,NaN,1479425445583437918.jpg
796,1.480000e+18,0.051884,1,NaN,NaN,1479425480989617990.jpg
2927,1.480000e+18,0.076301,0,NaN,NaN,1479425587558231155.jpg
1652,1.480000e+18,0.031416,0,NaN,NaN,1479425523796982853.jpg
2914,1.480000e+18,0.000348,1,NaN,NaN,1479425586908125583.jpg


In [8]:
# define path variables
parent_path = os.path.dirname(os.getcwd())

data_path = os.path.join(parent_path, 'dataset')
img_front_dir_path = os.path.join(dataset)

model_path = os.path.join(parent_path, 'model')
log_path = os.path.join(model_path, 'log')

csv_dir_path = os.path.join(data_path, 'csv', 'final')
cur_file = 'C:/Users/Ketan/Thesis/self-driving-car/challenges/challenge-2/out.csv'

In [9]:
def img_to_arr(p):
    with image.load_img(p) as img:
        img = image.img_to_array(img)
    return img

# values computed from dataset sample.
# def normalize(img):
#     img[:,:,0] -= 94.9449
#     img[:,:,0] /= 58.6121

#     img[:,:,1] -= 103.599
#     img[:,:,1] /= 61.6239

#     img[:,:,2] -= 92.9077
#     img[:,:,2] /= 68.66
    
#     return img

In [10]:
# define generator that loops through the data
def generator(df, batch_size, img_shape, should_shuffle):
    # shuffle dataframe for each epoch
    if should_shuffle:
        df = shuffle(df)
        
    img_list = df['File'].values
    wheel_axis = df['angle'].values
    
    # create empty batch
    batch_img = np.zeros((batch_size,) + img_shape)
    batch_label = np.zeros((batch_size, 1))
    
    index = 0
    while True:
        for i in range(batch_size):
            img_name = img_list[index]
            arr = img_to_arr(os.path.join(img_front_dir_path, img_name))
            
            batch_img[i] = arr
            batch_label[i] = wheel_axis[index] 
            
            index += 1
            if index == len(img_list):
                index = 0
            
        yield batch_img, batch_label

In [11]:
df_train.File.iloc[0]

'1479425445583437918.jpg'

In [12]:
#input_shape = img_to_arr(os.path.join(img_front_dir_path, df_train['File'])).shape
input_shape = (480, 640, 3)
batch_size = 16
train_steps = (df_train.shape[0] / batch_size) + 1
val_steps = (df_val.shape[0] / batch_size) + 1

print(" batch_size: %d, train_steps: %d, val_steps: %d" % 
      ( batch_size, train_steps, val_steps))

 batch_size: 16, train_steps: 281, val_steps: 71


In [13]:
train_batch = generator(df_train, batch_size, input_shape, True)
val_batch = generator(df_val, batch_size, input_shape, False)

In [ ]:
# define PilotNet model, with batch normalization included.
def get_model(input_shape):
    model = Sequential([
        Conv2D(24, kernel_size=(5,5), strides=(2,2), activation='relu', input_shape=input_shape),
        BatchNormalization(axis=1),
        Conv2D(36, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(48, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(axis=1),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(50, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='relu'),
        BatchNormalization(),
        Dense(1)
    ])
    
    return model

model = get_model(input_shape)
sgd = SGD(learning_rate=1e-3, decay=1e-4, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss="mse") 
model.summary()

In [14]:
# define callbacks
cur_model = cur_file + '-PilotNet_v2'
csv_logger = CSVLogger(os.path.join(log_path, cur_model + '.log'))

model_file_name= os.path.join(model_path, cur_model + '-{epoch:03d}-{val_loss:.5f}.h5')
checkpoint = ModelCheckpoint(model_file_name, verbose=0, save_best_only=True)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [15]:
checkpoint_filepath = './logs1'


In [16]:
callback1 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_loss', verbose=0, 
                                               save_best_only=False, save_weights_only=False, mode='auto', 
                                               save_freq='epoch',options=None)

In [ ]:
with tf.device('/GPU:0'):

    model.fit_generator(train_batch, 
                        train_steps, 
                        epochs=5, 
                        verbose=1, 
                        callbacks=[csv_logger, callback1, early_stop], 
                        validation_data=val_batch, 
                        validation_steps=val_steps, 
                        initial_epoch=0)

In [ ]:
model.save_weights('udacity_regression_model_weights_v1.h5')

In [ ]:
model.save('udacity_regression_model_v1.h5')

In [27]:
from keras.applications import inception_v3 as inc_net


In [28]:
def transform_img_fn(path_list):
    out = []
    for img_path in path_list:
        img = image.load_img((img_path), target_size=(480,640))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = inc_net.preprocess_input(x)
        out.append(x)
    return np.vstack(out) /2 +0.5

In [31]:
images = transform_img_fn([os.path.join(r'C:/Users/Ketan/Thesis/self-driving-car/challenges/challenge-2/','1479425441432724303.jpg')])


In [ ]:
model.predict(images)

In [17]:
from tensorflow.keras.applications.resnet50 import ResNet50


In [18]:
base_model = ResNet50(weights='imagenet',input_shape=(480,640,3),include_top=False)


In [19]:
base_model.trainable = False

In [20]:
model1 = Sequential()
# model.add(InputLayer(input_shape=(128,380, 3)))
model1.add(base_model)
model1.add(pooling.MaxPooling2D())
model1.add(Flatten())
model1.add(Dense(256, activation='relu'))
# model1.add(Dropout(.5))
model1.add(Dense(128, activation='relu'))
# model.add(core.Dropout(.5))
# model.add(core.Dense(10, activation='relu'))
model1.add(Dense(1, activation='relu'))
sgd = SGD(learning_rate=1e-3, decay=1e-4, momentum=0.9, nesterov=True)

model1.compile(optimizer=sgd, loss="mse") 


In [ ]:
model1.summary()

In [22]:
with tf.device('/GPU:0'):

    model1.fit_generator(train_batch, 
                        train_steps, 
                        epochs=1, 
                        verbose=1, 
                        callbacks=[csv_logger, callback1, early_stop], 
                        validation_data=val_batch, 
                        validation_steps=val_steps, 
                        initial_epoch=0)

281/281 [==============================] - 816s 3s/step - loss: 0.0432 - val_loss: 0.0426


C:\Users\Ketan\anaconda3\envs\tf-gpu-cuda\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: .\logs1\assets


In [25]:
model1.save_weights('udacity_tl_regression_weights_v1.h5')

In [26]:
model1.save('udacity_tl_regression_v1.h5')

In [32]:
model1.predict(images)

array([[0.]], dtype=float32)